In [9]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git

In [10]:
!pip install -q datasets bitsandbytes einops wandB

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00004-of-00014.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [16]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [18]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [33]:
import json
from datasets import Dataset

def load_paragraph_list(input_file):
    with open(input_file, 'r') as file:
        paragraph_list = json.load(file)
    return paragraph_list

def convert_to_dataset(paragraph_list):
    dataset_dict = {
        "input_text": [],
        "target_text": []
    }

    for paragraph in paragraph_list:
        dataset_dict["input_text"].append(paragraph["input"])
        dataset_dict["target_text"].append(paragraph["target"])

    dataset = Dataset.from_dict(dataset_dict)
    return dataset

# Example usage
input_file = 'list.txt'
paragraph_list = load_paragraph_list(input_file)
dataset1 = convert_to_dataset(paragraph_list)
dataset1

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 63657
})

In [34]:
from datasets import load_dataset

dataset_name = 'findnitai/english-to-hinglish' # Roman Urdu OR Hingllish
dataset = load_dataset(dataset_name, split="train")

def remove_source(example):
    return {
        'input_text': example['translation']['en'],
        'target_text': example['translation']['hi_ng']
    }

dataset = dataset.map(remove_source, remove_columns=['translation'])
dataset

In [39]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset1,
    peft_config=peft_config,
    dataset_text_field="input_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()

Map:   0%|          | 0/63657 [00:00<?, ? examples/s]

RuntimeError: MPS backend out of memory (MPS allocated: 20.23 GB, other allocations: 1.70 MB, max allowed: 20.40 GB). Tried to allocate 172.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).